In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [2]:
batchsize=8

In [3]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2, shear_range=0.2, zoom_range=0.2,
                               width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)

train_data=train_datagen.flow_from_directory(r'C:\Drwsiness\Prepared_Data\train',target_size=(80,80),
                                             batch_size=batchsize, class_mode='categorical',subset='training')

validation_data= train_datagen.flow_from_directory(r'C:\Drwsiness\Prepared_Data\train',target_size=(80,80),
                                             batch_size=batchsize, class_mode='categorical',subset='validation')

Found 64530 images belonging to 2 classes.
Found 16132 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r'C:\Drwsiness\Prepared_Data\test',target_size=(80,80),
                                             batch_size=batchsize, class_mode='categorical')


Found 4236 images belonging to 2 classes.


In [5]:
bmodel = InceptionV3(include_top=False, weights ='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model (inputs = bmodel.input, outputs = hmodel)
for layer in bmodel.layers:
    layer.trainable = False


In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [7]:
checkpoint = ModelCheckpoint(r'C:\Drwsiness\models\model.h5',monitor = 'val_loss', save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7,verbose =3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor = 'val_loss', patience=3,verbose=3,)

callbacks = [checkpoint,earlystop,learning_rate]


In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs =50)

C:\Users\Moinaktar shaikh\AppData\Local\Temp\ipykernel_18408\716146542.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/50
8065/8066 [============================>.] - ETA: 0s - loss: 0.2084 - accuracy: 0.9187
Epoch 1: val_loss improved from inf to 0.19547, saving model to C:\Drwsiness\models\model.h5
8066/8066 [==============================] - 344s 42ms/step - loss: 0.2084 - accuracy: 0.9187 - val_loss: 0.1955 - val_accuracy: 0.9171 - lr: 0.0010
Epoch 2/50
8066/8066 [==============================] - ETA: 0s - loss: 0.1768 - accuracy: 0.9314
Epoch 2: val_loss did not improve from 0.19547
8066/8066 [==============================] - 337s 42ms/step - loss: 0.1768 - accuracy: 0.9314 - val_loss: 0.2034 - val_accuracy: 0.9154 - lr: 0.0010
Epoch 3/50
8065/8066 [============================>.] - ETA: 0s - loss: 0.1739 - accuracy: 0.9331
Epoch 3: val_loss improved from 0.19547 to 0.19425, saving model to C:\Drwsiness\models\model.h5
8066/8066 [==============================] - 334s 41ms/step - loss: 0.1739 - accuracy: 0.9331 - val_loss: 0.1942 - val_accuracy: 0.9190 - lr: 0.0010
Epoch 4/50
8066/8066 [

In [9]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\Moinaktar shaikh\AppData\Local\Temp\ipykernel_18408\713691994.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.13296109437942505
0.949573814868927


In [10]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)


C:\Users\Moinaktar shaikh\AppData\Local\Temp\ipykernel_18408\2510674040.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.17611336708068848
0.931006669998169


In [11]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)


C:\Users\Moinaktar shaikh\AppData\Local\Temp\ipykernel_18408\1566569169.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.13296109437942505
0.949573814868927
